# Import packages

In [ ]:
import os
import re
import unicodedata
# %pip install spacy --quiet
import spacy
# %pip install pandas --quiet
import pandas as pd
# %pip install matplotlib --quiet
import matplotlib.pyplot as plt
# %pip install python-dotenv --quiet
from dotenv import load_dotenv
# %pip install sklearn --quiet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
# %pip install nltk --quiet
import nltk
from nltk.stem import WordNetLemmatizer
# !python -m spacy download en_core_web_sm

# Load data

In [ ]:
# Load the environment variables
load_dotenv()

# Get the directory of the datasets
data_dir = os.getenv("DATA_DIR")

# Get the directory of the articles
articles_dir = os.path.join(data_dir, "articles")

In [ ]:
# Load the main dataset
df = pd.read_csv(f"{data_dir}/article_info_V2.csv", index_col=0, parse_dates=[1])

# Load the external dataset
external_df = pd.read_csv(f"{data_dir}/other-articles.csv", index_col=0, parse_dates=[1], sep=";")

# Load the custom categories dataset
custom_categories = pd.read_csv(f"{data_dir}/new-categories.csv")

In [ ]:
# Load the English spaCy model
nlp = spacy.load("en_core_web_sm")

# Load the stopwords from spacy
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load NLTK punkt package
nltk.download("punkt")

# Load NLTK wordnet package
nltk.download("wordnet")

# Load NLTK omw package
nltk.download("omw-1.4")

In [ ]:
df.info()

# Merge preparation

In [ ]:
# Drop unused columns
df.drop(columns=["Author", "Type", "Keywords"], inplace=True)

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Remove the url column from the external dataset
external_df.drop(columns=["Url"], inplace=True)

# Merge external dataset

In [ ]:
# Add the external dataset to the main dataset
df = pd.concat([df, external_df])

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Delete the external dataset variable
del external_df

# Cleaning

In [ ]:
# Remove duplicate entries by title and date
df.drop_duplicates(subset=["Title", "Date"], keep="first", inplace=True)

# Drop the date column
df.drop(columns=["Date"], inplace=True)

# After receiving feedback from SKC, we decided to drop the following categories:
custom_categories.drop(columns=["People", "Places", "Criminal groups"], inplace=True)

# Load the contents of articles

In [ ]:
# Article loading function
def load_article(title):
    # Create filename from title
    file_name = f"{title}.txt"
    # Get the path of the article
    file_path = os.path.join(articles_dir, file_name)
    try:
        # Attempt to read the article
        file = open(file_path, "r", encoding="utf-8")
    except:
        # If the article could not be read, return None
        return None
    # Read the contents of the article
    contents = file.read()
    # Close the file
    file.close()
    # Return the contents of the article
    return contents

# Load the articles
df["Contents"] = df["Title"].apply(load_article)

# Delete the article loading function
del load_article

# Remove articles which cannot be found

In [ ]:
# Remove rows without article contents
df.dropna(subset=["Contents"], inplace=True)

# Reset the index of the dataframe
df.reset_index(drop=True, inplace=True)

# Parse the old target

In [ ]:
# Define functions to parse target
def clean_string(str):
    # Make the string lowercase
    str = str.lower()
    # Remove all non-alphanumeric characters
    str = re.sub(r"[^\w\s]", "", str)
    # Remove the leading and trailing spaces
    return str.strip()

def parse_tags(str):
    # Split the string on each comma
    raw_list = str.split(",")
    # Clean every string in the list
    return list(map(clean_string, raw_list))


def get_unique_values(list_of_lists):
    # Get a list of unique tags from a list of lists
    unique_values = list(set([value for value_list in list_of_lists for value in value_list]))
    # Filter out any values that are empty
    unique_values = [tag for tag in unique_values if tag]
    # Return the unique values
    return unique_values

# Convert the raw string values of the Tags column to lists of strings
df["Tags"] = df["Tags"].apply(parse_tags)

# Get all the unique tags from the Tags column
unique_tags = get_unique_values(df["Tags"])

# Create a list of words which should not occur in the tags
forbidden_words = ["COVERAGE", "SEE", "INSIGHT CRIME", "SOURCES", "THE", "LINKED", "MEDIA", "CREATED", "HOTSPOT", "CARGO", "MORE"]

# Remove any tag that contains forbidden words
unique_tags = [tag for tag in unique_tags if not any(word in tag for word in forbidden_words)]

# Remove any tag that contains less than 3 characters
unique_tags = [tag for tag in unique_tags if len(tag) > 3]

# Remove any tag that contains more than 3 words
unique_tags = [tag for tag in unique_tags if len(tag.split()) <= 3]

print(unique_tags)

# Delete the variables that are no longer needed
del clean_string, parse_tags, forbidden_words, get_unique_values

# (Temporary) Validate all the remaining tags using the InSight Crime website

In [ ]:
# import requests

# # Make all unique tags lowercase
# unique_tags = [tag.lower() for tag in unique_tags]
# # Replace all non-ascii characters with ascii equivalents
# unique_tags = [unicodedata.normalize("NFKD", tag).encode("ascii", "ignore").decode("ascii") for tag in unique_tags]
# # Join the words in every unique tag together with dashes
# unique_tags = ["-".join(tag.split()) for tag in unique_tags]
# # Create a url using the unique tags and https://insightcrime.org/tag/ as the prefix
# urls = ["https://insightcrime.org/tag/" + tag for tag in unique_tags]
# # Send a GET request to each of the urls
# for url in urls:
#     # Send a GET request to the tag
#     response = requests.get(url)
#     # Print the current tag and response code to the console
#     print(f"{url} - {response.status_code}")
#     # If the response is 404, then remove the tag from the list
#     if response.status_code == 404:
#         # Remove the prefix from the url
#         tag = url.replace("https://insightcrime.org/tag/", "")
#         # Remove the unique tag
#         unique_tags.remove(tag)

# # Advice: Replace tags that are similar by the valid tag - e.g. paraguay news, paraguay -> paraguay

# Create the new target

In [ ]:
def increment_counters(counters, category_name):
    # Check if the category is in the counters dictionary
    if category_name not in counters:
        # If the category does not exist in the counters, create it
        counters[category_name] = 1
    else:
        # If the category exists in the counters, increment the counter
        counters[category_name] += 1

def iterate_categories(counters, tag):
    # Iterate over the categories
    for category_name in categories:
        # Get the current category
        category = categories[category_name]
        # Check if the tag is in the category list
        if tag in category:
            # Increment the counter of the category
            increment_counters(counters, category_name)

# Define a function to parse the custom categories into a list of strings
def load_category(name):
    try:
        # Load the category as a list while removing empty values
        return custom_categories[name].dropna().tolist()
    except:
        return None

# Initiate the category dictionary
categories = {}

# Iterate over these custom_categories columns
for category_name in custom_categories.columns:
    # Load the category
    categories[category_name] = load_category(category_name)

# Create a new column for the category
df["Category"] = ""

# Iterate over the dataframe to create the category for each article
for index, row in df.iterrows():
    # Get the tags of the current row
    tag_list = row["Tags"]

    # Create a dictionary for the counters of the categories for this article
    counters = {}

    # Iterate over every tag in the tag list
    for tag in tag_list:
        iterate_categories(counters, tag)

    # Check if any category was found for this article
    if len(counters) == 0:
        # If no matching category was found, assign the default category to the row
        row["Category"] = "Other"
    else:
        # Get the category with the highest counter and assign it to the row
        row["Category"] = max(counters, key=counters.get)

# Delete the variables that are no longer needed
del load_category

# Drop the old target column

In [ ]:
# Removes the tags column from the dataframe
df.drop(columns=["Tags"], inplace=True)

# Remove articles with the "Other" (default) category

In [ ]:
# Drop rows with the other category
df.drop(df[df["Category"] == "Other"].index, inplace=True)

# Visualizations

In [ ]:
category_counts = df["Category"].value_counts()
plt.figure(figsize=(12, 8))
plt.title("Category distribution")
plt.ylabel("Number of articles")
plt.bar(category_counts.index, category_counts.values)
plt.show()

# Prepare input data

In [ ]:
lemmatizer = WordNetLemmatizer()

# Lemmatize the words in sentences
def lemmatize_sentences(sentences):
    # Create a new list of lemmatized sentences
    cleaned_sentences = []
    # Iterate over every sentence
    for sentence in sentences:
        # Convert the sentence tokens
        words = nltk.word_tokenize(sentence)
        # Remove digits from the words
        words = [word for word in words if not word.isdigit()]
        # Remove stopwords from the words
        words = [word for word in words if word not in stop_words]
        # Lemmatize the words
        words = [lemmatizer.lemmatize(word) for word in words]
        # Remove words with less than 4 characters
        words = [word for word in words if len(word) > 3]
        # Check if the number of words is greater than 0
        if len(words) > 0:
            # Add the words to the list of cleaned sentences
            cleaned_sentences.append(words)
    # Return the cleaned sentences
    return cleaned_sentences

def text_to_sentences(text):
    # Make text lowercase
    text = text.lower()
    # Replace non ascii characters with their ascii counterpart
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")
    # Remove digits from the text
    text = re.sub(r"\d+", "", text)
    # Remove words with non-alphanumeric characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Remove double whitespace
    text = re.sub(r"\s+", " ", text)
    # Split the text into sentences
    sentences = nltk.sent_tokenize(text)
    # Lemmatize the sentences
    sentences = lemmatize_sentences(sentences)
    # Remove empty sentences
    sentences = list(filter(None, sentences))
    # Join the words in sentences together as a string
    sentences = list(map(lambda x: " ".join(x), sentences))
    # Return the list of sentences
    return sentences

# Create a new column for the sentences of the articles
df["Sentences"] = df["Contents"].apply(text_to_sentences)

# Delete variables that are no longer needed
del lemmatizer

# Convert input data to a TfidfVector

In [ ]:
# Check if a column/word occurs in at least 20 articles and in max 50% of the articles
def check_column(column):
    count = column.value_counts()[column.value_counts() > 0].count()
    return count >= 20 and count / len(column) < 0.5

# Create a vocabulary of words from the sentences
documents = df["Sentences"].apply(lambda x: " ".join(x)).tolist()

# Create a new TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Create a TFIDF matrix from the sentences
values = tfidf_vectorizer.fit_transform(documents)

# Get the column names of the TFIDF matrix
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a new dataframe from the TFIDF matrix
vector_df = pd.DataFrame(values.toarray(), columns=feature_names)

# Get columns that do not match the criteria of the check_column function
columns_to_drop = list(filter(lambda x: not check_column(vector_df[x]), vector_df.columns))

# Remove all columns that did not match the criteria
vector_df.drop(columns=columns_to_drop, inplace=True)

# Modelling

In [ ]:
# Some functions to display metrics of a model's performance
def specificity_score(target, predictions):
    return 1 - recall_score(target, predictions, average="weighted")

def get_metrics(target, predictions):
    return {
        "recall": recall_score(target, predictions, average="weighted"),
        "precision": precision_score(target, predictions, average="weighted"),
        "accuracy": accuracy_score(target, predictions),
        "specificity": specificity_score(target, predictions),
        "f1": f1_score(target, predictions, average="weighted"),
    }

def print_metrics(target, predictions):
    metrics = get_metrics(target=target, predictions=predictions)
    for metric in metrics:
        str_metric = "{:.2f}".format(metrics[metric] * 100)+"%"
        print(f"{metric}: {str_metric}")

# Split train and test data

In [ ]:
# Get input data
X = vector_df
# Get target data
y = pd.get_dummies(df["Category"])

# Split the data into training, testing and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

# Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create a DecisionTreeClassifier model
dtc = DecisionTreeClassifier(max_depth=50, random_state=42)

# Fit and train the model
dtc = dtc.fit(X_train, y_train)

In [ ]:
# Predict the test data
y_pred_dtc = dtc.predict(X_test)

# Predict the training data
y_pred_dtc_train = dtc.predict(X_train)

# Predict the validation data
y_pred_dtc_val = dtc.predict(X_val)

# Print the metrics of the test data
print("Test data:")
print_metrics(y_test, y_pred_dtc)

# Print the metrics of the training data
print("\nTrain data:")
print_metrics(y_train, y_pred_dtc_train)

# Print the metrics of the validation data
print("\nValidation data:")
print_metrics(y_val, y_pred_dtc_val)

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a DecisionTreeClassifier model
rfc = RandomForestClassifier()

# Fit and train the model
rfc = rfc.fit(X_train,y_train)

In [ ]:
# Predict the test data
y_pred_rfc = rfc.predict(X_test)

# Predict the training data
y_pred_rfc_train = rfc.predict(X_train)

# Predict the validation data
y_pred_rfc_val = rfc.predict(X_val)

# Print the metrics of the test data
print("Test data:")
print_metrics(y_test, y_pred_rfc)

# Print the metrics of the training data
print("\nTrain data:")
print_metrics(y_train, y_pred_rfc_train)

# Print the metrics of the validation data
print("\nValidation data:")
print_metrics(y_val, y_pred_rfc_val)

# K-Neighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Create the KNeighborsClassifier model
knn = KNeighborsClassifier()

# Fit and train the model
knn = knn.fit(X_train,y_train)

In [ ]:
# Predict the test data
y_pred_knn = knn.predict(X_test)

# Predict the training data
y_pred_knn_train = knn.predict(X_train)

# Predict the validation data
y_pred_knn_val = knn.predict(X_val)

# Print the metrics of the test data
print("Test data:")
print_metrics(y_test, y_pred_knn)

# Print the metrics of the training data
print("\nTrain data:")
print_metrics(y_train, y_pred_knn_train)

# Print the metrics of the validation data
print("\nValidation data:")
print_metrics(y_val, y_pred_knn_val)

In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

# Create a new column called Keywords
df["Keywords"] = ""

# Write multiple patterns to find if an article contains cocaine etc.
pattern = [
# Drugs
[{"LEMMA": "drug"},{"POS": "NOUN", "OP": "?"}],
[{"POS": "ADJ", "OP": "?"},{"TEXT": "cocaine"},{"POS": "NOUN", "OP": "?"}],
# Environmental crimes
[{"LEMMA": "deforestation"}],
[{"LEMMA": "environment"}],
[{"LEMMA": "illegal"},{"POS": "NOUN", "OP": "?"}],
# Violence
[{"LEMMA": "violence"}],
[{"TEXT": "violent"},{"POS": "NOUN", "OP": "?"}],
[{"LEMMA": "kidnap"}],
[{"POS": "NOUN", "OP": "?"},{"LEMMA": "threat"}],
[{"LEMMA": "murder"}],
[{"LEMMA": "shoot"}],
[{"TEXT": "death"}],
[{"TEXT": "assassin"}],
[{"LEMMA": "robbery"}],
[{"TEXT": "hitman"}],
[{"TEXT": "dead"}],
[{"TEXT": "execution"}],
[{"POS": "NOUN", "OP": "?"},{"LEMMA": "killing"}],
# Weapons
[{"POS": "ADJ", "OP": "?"},{"LEMMA": "weapon"},{"POS": "NOUN", "OP": "?"}],
[{"POS": "VERB", "OP": "?"},{"LEMMA": "weapon"}],
[{"LEMMA": "arms"},{"POS": "NOUN", "OP": "?"}],
[{"LEMMA": "gun"},{"POS": "NOUN", "OP": "?"}],
[{"TEXT": "AK-47"}],
[{"LEMMA": "explosive"}],
[{"POS": "NOUN", "OP": "?"},{"LEMMA": "rifle"}],
# Trafficking
[{"POS": "ADJ", "OP": "?"},{"LEMMA": "human"},{"POS": "NOUN", "OP": "?"}],
[{"POS": "NOUN", "OP":"?"},{"LEMMA": "trafficking"}],
[{"TEXT": "traffickers"}],
# Money
[{"POS": "ADJ", "OP": "?"},{"LEMMA": "laundering"},{"POS": "NOUN", "OP": "?"}],
[{"TEXT": "fraudulent"}],
[{"LEMMA": "fraude"}],
[{"LEMMA": "money"}],
[{"LEMMA": "corruption"}],
# Other
]

# Add the pattern to the matcher and 
matcher.add("DETERMINE CATEGORY", [pattern][0])

# Set the index of the article content
i = 0
# Loop over all articles contents
for content in df["Contents"][0:10]:
    doc = nlp(content)
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create an empty array for all keywords per article
    all_generated_keywords = []
    # Iterate over the matches and append them to an array
    for match_id, start, end in matches:
        all_generated_keywords.append(doc[start:end].text)
    # Remove duplicates from array and assign them to variable
    duplicates_removed =  list(dict.fromkeys(all_generated_keywords))
    # Append keywords without duplicates to the Keywords column in the dataframe
    df["Keywords"][i] = duplicates_removed
    # Increase the index of the article content
    i += 1

In [ ]:
# Check if a column/word occurs in at least 20 articles and in max 50% of the articles
def check_column(column):
    count = column.value_counts()[column.value_counts() > 0].count()
    return count >= 20 and count / len(column) < 0.5

# Create a vocabulary of words from the sentences
documents = df["Keywords"].apply(lambda x: " ".join(x)).tolist()

# Create a new TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Create a TFIDF matrix from the sentences
values = tfidf_vectorizer.fit_transform(documents)

# Get the column names of the TFIDF matrix
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a new dataframe from the TFIDF matrix
new_vector_df = pd.DataFrame(values.toarray(), columns=feature_names)

# Drop rows that contain 0 values in all columns
new_vector_df = new_vector_df.loc[new_vector_df.apply(lambda x: sum(x) > 0, axis=1)]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create a DecisionTreeClassifier model
dtc_spacy = DecisionTreeClassifier(random_state=42)

# Fit and train the model
dtc_spacy = dtc_spacy.fit(X_train, y_train)

In [ ]:
# Predict the test data
y_pred_spacy = dtc_spacy.predict(X_test)

# Predict the training data
y_pred_spacy_train = dtc_spacy.predict(X_train)

# Predict the validation data
y_pred_spacy_val = dtc_spacy.predict(X_val)

# Print the metrics of the test data
print("Test data:")
print_metrics(y_test, y_pred_spacy)

# Print the metrics of the training data
print("\nTrain data:")
print_metrics(y_train, y_pred_spacy_train)

# Print the metrics of the validation data
print("\nValidation data:")
print_metrics(y_val, y_pred_spacy_val)

In [ ]:
# Get all the metrics of the test data
test_metrics = {
    "DecisionTreeClassifier": get_metrics(y_test, y_pred_dtc),
    "RandomForestClassifier": get_metrics(y_test, y_pred_rfc),
    "KNeighborsClassifier": get_metrics(y_test, y_pred_knn),
    "SpacyDTC": get_metrics(y_test, y_pred_spacy),
}

# Get all the metrics of the training data
train_metrics = {
    "DecisionTreeClassifier": get_metrics(y_train, y_pred_dtc_train),
    "RandomForestClassifier": get_metrics(y_train, y_pred_rfc_train),
    "KNeighborsClassifier": get_metrics(y_train, y_pred_knn_train),
    "SpacyDTC": get_metrics(y_train, y_pred_spacy_train),
}

# Get all the metrics of the validation data
val_metrics = {
    "DecisionTreeClassifier": get_metrics(y_val, y_pred_dtc_val),
    "RandomForestClassifier": get_metrics(y_val, y_pred_rfc_val),
    "KNeighborsClassifier": get_metrics(y_val, y_pred_knn_val),
    "SpacyDTC": get_metrics(y_val, y_pred_spacy_val),
}

In [ ]:
# Visualize the f1 score of the test, train and validation metrics
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(["DecisionTreeClassifier", "RandomForestClassifier", "KNeighborsClassifier", "Spacy DTC"], [test_metrics["DecisionTreeClassifier"]["f1"], test_metrics["RandomForestClassifier"]["f1"], test_metrics["KNeighborsClassifier"]["f1"], test_metrics["SpacyDTC"]["f1"]], label="Test")
ax.plot(["DecisionTreeClassifier", "RandomForestClassifier", "KNeighborsClassifier", "Spacy DTC"], [train_metrics["DecisionTreeClassifier"]["f1"], train_metrics["RandomForestClassifier"]["f1"], train_metrics["KNeighborsClassifier"]["f1"], train_metrics["SpacyDTC"]["f1"]], label="Train")
ax.plot(["DecisionTreeClassifier", "RandomForestClassifier", "KNeighborsClassifier", "Spacy DTC"], [val_metrics["DecisionTreeClassifier"]["f1"], val_metrics["RandomForestClassifier"]["f1"], val_metrics["KNeighborsClassifier"]["f1"], val_metrics["SpacyDTC"]["f1"]], label="Validation")
ax.set_xlabel("Classifier")
ax.set_ylabel("F1 Score")
ax.set_title("F1 Score of the test, train and validation data of each model")
ax.legend()
plt.show()